# Gerenciador de investimentos

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
# from lib import lib as l
from lib import plot as p
from lib import data as d
from lib import finance as f

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA', '#27FEFA'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2', '#162CF2'])

## Leitura dos dados

In [3]:
dinv, dport, dport_inv, dprices = f.read_invest()
dinvest = pd.concat([dinv, dport_inv]).reset_index(drop=True)

## Fluxos de investimentos

### Histórico de fluxos

In [4]:
dflow = dinvest.groupby('ID').apply(f.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow['Rend'] = dflow['Rendimento anual'].apply(d.pct2int)
dflow = dflow.sort_values(by=['Rend'], ascending=False).reset_index(drop=True)
a = dflow.pop('Rend')

### Fluxos ativos

In [5]:
dflow[dflow['Ativo']]

,ID,Valor inicial,Ganho,Meses,Rendimento,Rendimento mensal,Rendimento anual,Ganho em 1 ano,Ativo,Tipo
0,100.000,1308.000,86.000,4.918,6.57%,1.3%,16.76%,233.634,True,IVVB11
4,14.300,930.000,2.030,0.623,0.22%,0.35%,4.28%,39.869,True,Nu
9,1.200,10337.670,39.850,1.279,0.39%,0.3%,3.67%,380.482,True,Nu
13,13.000,10000.000,32.800,1.279,0.33%,0.26%,3.11%,312.237,True,Nu
19,15.000,3000.000,1.920,0.492,0.06%,0.13%,1.57%,47.101,True,Nu
23,102.000,1390.300,3.700,3.508,0.27%,0.08%,0.91%,12.696,True,IVVB11
31,109.000,6.700,0.000,1.836,0.0%,0.0%,0.0%,0.000,True,Cash
32,105.000,1478.000,-84.000,1.869,-5.68%,-3.08%,-31.25%,-435.617,True,IVVB11
33,101.000,875.000,-256.500,4.426,-29.31%,-7.54%,-60.86%,-376.410,True,BRAX11
34,103.000,957.200,-313.400,2.590,-32.74%,-14.2%,-84.0%,-540.784,True,BBSD11


## Performance

In [6]:
dnu = dflow[dflow['Meses'] > 0]
dnu = dnu[dnu['Tipo'] == 'Nu']

detf = dflow[dflow['Meses'] > 0]
detf = detf[detf['Tipo'] != 'Nu']

### Renda fixa

In [7]:
p.change_df_prop(f.calc_flows_overview(dnu), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
24344.3,273.66,0.28%,779.689,3.2%


### Renda Variável

In [8]:
p.change_df_prop(f.calc_flows_overview(detf), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
7975.6,-1834.8,-18.7%,-3523.66,-44.18%


### Mensal

In [9]:
fig_prof = p.make_fig()
fig_prof_total = p.make_fig()

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [10]:
dret = d.pivot_agg(f.calc_flow_gains(dinvest[dinvest['Tipo'] != 'Cash']), val='ValorDelta', idx='Data', col='Tipo', agg={'ValorDelta':np.sum})
dret['Portfolio'] = dret.sum(axis=1)
cols = dret.columns[1:]
for c in cols:
    dret = d.window(dret, val=c)

In [11]:
palette1 = {'Portfolio': lcolors[0], 'BRAX11': lcolors[2], 'IVVB11': lcolors[0], 'BBSD11': lcolors[1], 'Cash': lcolors[4]}
palette2 = {'PortfolioAvg': dcolors[0], 'BRAX11Avg': dcolors[2], 'IVVB11Avg': dcolors[0], 'BBSD11Avg': dcolors[1], 'Cash': dcolors[4]}
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette1, y=dret.columns[5:6], plot_type='bar')
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette2, y=dret.columns[10:11], name=['Portfolio médio'])

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …